In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
from tqdm import tqdm
import re

ACT_DDD_ROOT = "https://www.whocc.no/atc_ddd_index"
ATC_DDD_URL = f"{ACT_DDD_ROOT}/?code="
ATC_RE = r"([\w\d]+)\s+\<b\>\<a\s?href\=\"\.(\/\?.+?)\">([\w\s\.\\\/\,\-]+)\<\/a\>"

In [2]:
def get_atc_l1_code():
    response = requests.get(ACT_DDD_ROOT)
    parsed = BeautifulSoup(response.text, 'lxml')
    content = str(parsed.select_one("#content > div:nth-child(5) > div:nth-child(2) > p").encode_contents())
    atc_dataset = re.findall(ATC_RE, content)
    return [(code, ACT_DDD_ROOT + href.replace("&amp;", "&"), name) for code, href, name in atc_dataset]


In [ ]:
atc_l1_code = get_atc_l1_code()
pd.DataFrame(atc_l1_code, columns=["code", "href", "name"]).to_excel('atc_l1.xlsx', index=False)

In [ ]:
def get_atc_ddd_data(target_list):
    temp_list = []
    for code in tqdm(target_list):
        response = requests.get(ATC_DDD_URL + code)
        parsed = BeautifulSoup(response.text, 'lxml')
    

In [ ]:
def get_atc_ddd_data(target_list, level):
    temp_list = []
    for code in tqdm(target_list):
        response = requests.get(ATC_DDD_URL + code)
        parsed = BeautifulSoup(response.text, 'lxml')
        a_element_list = [b_element.find('a') for b_element in parsed.findAll('b')]
        row_data_list = a_element_list[:level] if level == 1 else a_element_list[level-1:]
        if row_data_list:
            for row_data in row_data_list:
                temp_list.append((row_data['href'].split('&')[0].split('=')[1], row_data.text,))
    return pd.DataFrame(temp_list, columns=['href', 'text'])

In [ ]:
atc_l1 = get_atc_ddd_data(string.ascii_uppercase, 1)
atc_l1.to_excel('ATC_L1.xlsx', index=False)

In [ ]:
atc_l1.to_excel('ATC_L1.xlsx', index=False)

In [ ]:
atc_l2 = get_atc_ddd_data(string.ascii_uppercase, 2)
atc_l2.to_excel('ATC_L2.xlsx', index=False)

In [ ]:
atc_l1 = get_atc_ddd_data(string.ascii_uppercase, 1)
atc_l1.to_excel('ATC_L1.xlsx', index=False)

In [ ]:
atc_l1.to_excel('ATC_L1.xlsx', index=False)

In [ ]:
atc_l2 = get_atc_ddd_data(string.ascii_uppercase, 2)
atc_l2.to_excel('ATC_L2.xlsx', index=False)

In [ ]:
atc_l2 = get_atc_ddd_data(string.ascii_uppercase, 2)
atc_l2.to_excel('ATC_L2.xlsx', index=False)

In [ ]:
atc_l1.to_excel('ATC_L1.xlsx', index=False)

In [ ]:
atc_l2 = get_atc_ddd_data(string.ascii_uppercase, 2)
atc_l2.to_excel('ATC_L2.xlsx', index=False)

In [ ]:
atc_l2 = get_atc_ddd_data(string.ascii_uppercase, 2)
atc_l2.to_excel('ATC_L2.xlsx', index=False)

In [ ]:
atc_l3 = get_atc_ddd_data(list(atc_l2['href']), 3)
atc_l3.to_excel('ATC_L3.xlsx', index=False)

In [ ]:
atc_l4 = get_atc_ddd_data(list(atc_l3['href']), 4)
atc_l4.to_excel('ATC_L4.xlsx', index=False)

In [ ]:
atc_l4_list = []
for i in tqdm(list(atc_l4['href'])):
    req = requests.get(ATC_DDD_URL+i)
    soup = BeautifulSoup(req.text, 'lxml')
    try:
        row4_df = pd.read_html(str(soup.select('ul table')), header=0)[0].fillna('')
        for n in row4_df.index:
            if row4_df.at[n, 'ATC code'] == '':
                try:
                    row4_df.at[n, 'ATC code'] = row4_df.at[n - 1, 'ATC code']
                except KeyError:
                    row4_df.at[n, 'ATC code'] = i
        atc_l4_list.extend(row4_df.to_dict('r'))
    except ValueError:
         atc_l4_list.extend([{'ATC code':i}])

In [ ]:
with pd.ExcelWriter('ATC_DDD.xlsx') as xlsx:
    atc_l1.to_excel(xlsx, sheet_name='ATC_L1', index=False)
    atc_l2.to_excel(xlsx, sheet_name='ATC_L2', index=False)
    atc_l3.to_excel(xlsx, sheet_name='ATC_L3', index=False)
    atc_l4.to_excel(xlsx, sheet_name='ATC_L4', index=False)
    pd.DataFrame(atc_l4_list).to_excel(xlsx, sheet_name='ATC_L5', index=False)